In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import display
import bcrypt



In [ ]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_ENTRADA = os.getenv("DB_ENTRADA")
DB_SAI = os.getenv("DB_SAI")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

ENTRADA = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_ENTRADA}"
SAIDA = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_SAI}'

engine_entrada = create_engine(ENTRADA)
engine_saida = create_engine(SAIDA)


query = "SELECT * FROM Administrador"
dados_originais_df = pd.read_sql(query, engine_entrada)
df_anon_admin = dados_originais_df.copy()


In [ ]:
def anonimizar_telefone(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anon_admin["telefone"] = df_anon_admin["telefone"].map(anonimizar_telefone)




In [ ]:
def anonimizar_cpf(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anon_admin["cpf"] = df_anon_admin["cpf"].map(anonimizar_cpf)


In [ ]:
def criptografar_senha(senha):
    
    salt = bcrypt.gensalt()
    hash_senha = bcrypt.hashpw(senha.encode('utf-8'), salt)
    return hash_senha.decode('utf-8')

def verificar_senha(senha_digitada, senha_criptografada):

    return bcrypt.checkpw(senha_digitada.encode('utf-8'), senha_criptografada.encode('utf-8'))

df_anon_admin["senha"] = df_anon_admin["senha"].apply(criptografar_senha)






In [ ]:

def anonimizar_email(email):
    email = str(email)
    parte_local, dominio = email.split("@")
  
    if len(parte_local) > 3:
        parte_local_pseudo = parte_local[:0] + "****"
    else:
        parte_local_pseudo = parte_local + "****"
    return parte_local_pseudo + "@" + dominio

df_anon_admin["email"] = df_anon_admin["email"].apply(anonimizar_email)


In [ ]:
df_anon_admin["nome_completo"] = list(map(
    lambda i: f"Administrador {i+1}", 
    range(len(df_anon_admin))
))




In [ ]:
dados_originais_df.to_csv('original_administrador.csv')
df_anon_admin.to_csv('pseudo_administrador.csv')

In [ ]:
df_anon_admin.to_sql(name='administrador', con=engine_saida, if_exists='replace', index=False)